In [41]:
import joblib
import numpy as np
import pandas as pd
import librosa
import os


In [42]:
model_queen_presence=joblib.load("queen_presence_model.pkl")
model_anomaly = joblib.load("bee_sound_anomaly_model.pkl")
model_queen_acceptance = joblib.load('queen_acceptance_rf_model.joblib')


In [43]:
# def extract_audio_features(file_path, n_mfcc=13):
#     y, sr = librosa.load(file_path, sr=22050)
#     mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
#     mfccs_mean = np.mean(mfccs.T, axis=0)
#     return mfccs_mean

# mfcc_features = extract_audio_features(audio_path, n_mfcc=13)

In [44]:
import librosa
import soundfile as sf
import pyaudio
import numpy as np
import tempfile
import os


def record_audio_and_extract_mfcc(sr,duration=10):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    p = pyaudio.PyAudio()
    stream = p.open(
        format=FORMAT, channels=CHANNELS, rate=sr, input=True, frames_per_buffer=CHUNK
    )

    print("Recording...")
    frames = []
    for _ in range(0, int(sr / CHUNK * duration)):
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    print("Recording complete.")

    audio_bytes = b"".join(frames)
    audio_np = np.frombuffer(audio_bytes, dtype=np.int16).astype(np.float32)
    max_val = np.max(np.abs(audio_np))
    if max_val > 0:
        audio_np /= max_val  # Normalize

    # Save temporarily with delete=False
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        tmp_name = tmp.name
        sf.write(tmp_name, audio_np, sr)

    # Now load with librosa
    y, _ = librosa.load(tmp_name, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0)

    # Clean up the temporary file manually
    os.remove(tmp_name)

    return mfccs_mean


In [45]:
# 0,0

audio_path = r"D:\AIDS\3rd year\sem2\Bee Hive Condition Projecct\archive\sound_files\split_audio\2022-06-25--15-04-12_1__segment1_part2.wav"
hive_temp = 22
hive_humidity = 49
weather_temp = 18
weather_humidity = 67

In [46]:
# # 0,0

# audio_path = r"D:\AIDS\3rd year\sem2\Bee Hive Condition Projecct\archive\sound_files\split_audio\2022-06-25--15-04-12_1__segment1_part2.wav"
# hive_temp = 34
# hive_humidity = 33
# weather_temp = 30
# weather_humidity = 37


# # audio_path = input("Enter the full path of the audio clip (.wav): ").strip('"')
# # hive_temp = float(input("Enter hive temperature: "))
# # hive_humidity = float(input("Enter hive humidity: "))
# # weather_temp = float(input("Enter weather temperature: "))
# # weather_humidity = float(input("Enter weather humidity: "))

In [54]:
audio_path = r"D:\AIDS\3rd year\sem2\Bee Hive Condition Projecct\archive\sound_files\split_audio\2022-06-25--15-04-12_1__segment2_part4.wav"
hive_temp = 34
hive_humidity = 33
weather_temp = 30
weather_humidity = 37

In [55]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import librosa

def extract_audio_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

def record_and_extract_mfcc(duration=10, sample_rate=22050, file_name="recorded_audio.wav", n_mfcc=13):
    print(f"Recording for {duration} seconds...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()
    wav.write(file_name, sample_rate, audio)
    print(f"Recording saved as {file_name}")
    
    mfcc_features = extract_audio_features(file_name, n_mfcc=n_mfcc)
    return mfcc_features

# Example usage:
mfcc = record_and_extract_mfcc()
print("Extracted MFCC features:", mfcc)

Recording for 10 seconds...
Recording saved as recorded_audio.wav
Extracted MFCC features: [-594.3562      85.34025    -10.010674   -13.021552    -3.703562
   -2.2824492   -9.435291    -4.748137    -8.6276245   -8.345287
  -13.361113    -9.478984    -8.010074 ]


In [56]:
def extract_audio_features(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

mfcc_features = extract_audio_features(audio_path, n_mfcc=13)
# mfcc_features = record_audio_and_extract_mfcc(22050, duration=10)
# mfcc_features = record_and_extract_mfcc()

mfcc_names = [f"mfcc_{i+1}" for i in range(len(mfcc_features))]
all_features = np.concatenate(([hive_temp, hive_humidity, weather_temp, weather_humidity], mfcc_features))
all_feature_names = ["hive temp", "hive humidity", "weather temp", "weather humidity"] + mfcc_names
input_df = pd.DataFrame([all_features], columns=all_feature_names)

In [57]:
mfcc_features

array([-282.03088  ,  164.14893  ,  -15.368212 ,   62.181347 ,
        -25.451149 ,   36.745518 ,  -14.543886 ,   16.210735 ,
         -7.420808 ,    0.9244495,    4.5058665,  -11.407895 ,
         10.53649  ], dtype=float32)

In [58]:
pred_qp = model_queen_presence.predict(input_df)[0]

# === OUTPUT ===
print("\n🧠 Predictions:")
print(f"🐝 Queen Presence: {'Yes' if pred_qp == 1 else 'No'}")


🧠 Predictions:
🐝 Queen Presence: No


In [59]:
if pred_qp==1:
    # randome forest model
    predictions = model_queen_acceptance.predict(input_df)

    print("Predicted Queen Acceptance:", predictions[0])

In [60]:
pred_anomaly = model_anomaly.predict(input_df)[0] 
if weather_temp >= 45 or hive_temp >= 45:
    pred_anomaly="Temp Rising"
    print(f"🚨 Anomaly Detected in Sound: anomaly")

else:
    print(f"🚨 Anomaly Detected in Sound: {'Yes' if pred_anomaly == -1 else 'No'}")


🚨 Anomaly Detected in Sound: No


In [61]:
print(f"🚨 Anomaly Detected in Sound: {'Yes' if pred_anomaly == -1 else 'No'}")

🚨 Anomaly Detected in Sound: No
